### Load config file

In [1]:
config_file_path = "../configs/config_model1.json"

In [2]:
import os
import sys
import pickle
import numpy as np
from pprint import pprint

sys.path.insert(0, "../")

from utils import load_json_config

# imports for displaying a video an IPython cell
import io
import base64
from IPython.display import HTML

In [3]:
# Load config file
config = load_json_config(config_file_path)

### Get predictions

In [4]:
prediction_file_pickle_path = os.path.join('../', config['output_dir'], config['model_name'], 'test_results.pkl')

In [5]:
with open(prediction_file_pickle_path, 'rb') as fp:
    logits_matrix, features_matrix, targets_list, item_id_list, class_to_idx = pickle.load(fp)

In [6]:
logits_matrix.shape

(24777, 174)

In [7]:
targets_list.shape

(24777,)

### Get confusion matrix

In [8]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [9]:
preds = logits_matrix.argmax(axis=1)
preds.shape

(24777,)

In [10]:
target_names = [class_to_idx[i] for i in range(int(len(class_to_idx) / 2))]

In [11]:
target_names

['Approaching something with your camera',
 'Attaching something to something',
 'Bending something so that it deforms',
 'Bending something until it breaks',
 'Burying something in something',
 'Closing something',
 'Covering something with something',
 'Digging something out of something',
 'Dropping something behind something',
 'Dropping something in front of something',
 'Dropping something into something',
 'Dropping something next to something',
 'Dropping something onto something',
 'Failing to put something into something because something does not fit',
 'Folding something',
 'Hitting something with something',
 'Holding something',
 'Holding something behind something',
 'Holding something in front of something',
 'Holding something next to something',
 'Holding something over something',
 'Laying something on the table on its side, not upright',
 'Letting something roll along a flat surface',
 'Letting something roll down a slanted surface',
 'Letting something roll up a sl

In [12]:
report = classification_report(targets_list, preds, target_names=target_names)

In [13]:
confusion_mat = confusion_matrix(targets_list, preds)

In [14]:
confusion_mat

array([[103,   0,   0, ...,   0,   0,   0],
       [  0,  20,   1, ...,   0,   0,   0],
       [  0,   0,  47, ...,   1,   1,   0],
       ...,
       [  0,   0,   1, ..., 202,  12,   0],
       [  0,   2,   0, ...,   5, 126,   0],
       [  0,   0,   0, ...,   1,   0,  65]])

In [15]:
confusion_mat.shape

(174, 174)

In [16]:
confusions = {}
for i in range(174):
    support = np.sum(confusion_mat[i])
    for j in range(174):
        if i != j:
            confusions['{}:{}'.format(i, j)] = (confusion_mat[i, j] / support) * 100

In [17]:
# sort confusions
import operator
confusions_sorted = sorted(confusions.items(), key=operator.itemgetter(1), reverse=True)

### Top-K confusions

In [18]:
K = 20

In [19]:
for i in range(K):
    elem = confusions_sorted[i]
    y_true = int(elem[0].split(":")[0])
    y_pred = int(elem[0].split(":")[1])
    print("{} --> {}:\t{:.2f}%".format(class_to_idx[y_true], class_to_idx[y_pred], elem[1]))

Moving something across a surface until it falls down --> Pushing something so that it falls off the table:	55.42%
Throwing something onto a surface --> Throwing something:	43.90%
Pushing something off of something --> Pushing something so that it falls off the table:	41.67%
Letting something roll along a flat surface --> Rolling something on a flat surface:	40.12%
Pouring something onto something --> Pouring something into something:	35.29%
Pouring something into something until it overflows --> Pouring something into something:	34.00%
Poking a stack of something without the stack collapsing --> Touching (without moving) part of something:	33.33%
Pulling something out of something --> Taking something out of something:	32.53%
Pretending to take something from somewhere --> Pretending to pick something up:	32.00%
Lifting a surface with something on it but not enough for it to slide down --> Tilting something with something on it slightly so it doesn't fall down:	31.58%
Poking something

### Top-K pretending confusions

In [20]:
k = 10
i = 0
while k > 0 and i < len(confusions_sorted):
    elem = confusions_sorted[i]
    y_true = class_to_idx[int(elem[0].split(":")[0])]
    y_pred = class_to_idx[int(elem[0].split(":")[1])]
    if ("Pretending" in y_true or "pretending" in y_true or
        "Pretending" in y_pred or "pretending" in y_pred):
        print("{} --> {}:\t{:.2f}%".format(y_true, y_pred, elem[1]))
        k -= 1
    i += 1
    

Pretending to take something from somewhere --> Pretending to pick something up:	32.00%
Pretending to put something behind something --> Putting something behind something:	30.43%
Pretending to poke something --> Poking something so lightly that it doesn't or almost doesn't move:	29.00%
Pretending to turn something upside down --> Turning something upside down:	22.67%
Pretending to squeeze something --> Squeezing something:	21.88%
Pretending or trying and failing to twist something --> Trying to bend something unbendable so nothing happens:	21.62%
Pretending or trying and failing to twist something --> Twisting something:	21.62%
Pretending to scoop something up with something --> Scooping something up with something:	19.23%
Pretending to close something without actually closing it --> Pretending to open something without actually opening it:	18.87%
Spilling something behind something --> Pretending to put something behind something:	17.65%
